# Teads challenge

In [1]:
import pandas
df = pandas.read_csv("aft100k.csv")
print(df.head(10))

   creative_id user_operating_system       user_device  \
0       113521               Android             Phone   
1       115340               Windows  PersonalComputer   
2       113582               Android             Phone   
3        97385               Windows  PersonalComputer   
4       114821               Windows  PersonalComputer   
5       113065               Android            Tablet   
6       111414                   iOS             Phone   
7       111414                   iOS             Phone   
8       112705               Windows  PersonalComputer   
9       113176                 macOS  PersonalComputer   

   average_seconds_played      cost  revenue  
0                     NaN  0.010128      0.0  
1                0.000000  0.005937      0.0  
2                7.142857  0.004398      0.0  
3                     NaN  0.006157      0.0  
4                     NaN  0.001994      0.0  
5                     NaN  0.003781      0.0  
6               17.000000  0.002

## Preeliminary questions

####  1) The margin being defined as (revenue - cost) / revenue, what is our global margin based on this log?

In [2]:
print("Are there null revenues? {}".format(df["revenue"].isnull().values.any()))
print("Are there null costs? {}".format(df["cost"].isnull().values.any()))


Are there null revenues? False
Are there null costs? False


In [3]:
revs = df["revenue"].sum()
costs = df["cost"].sum()
margin = (revs-costs)/revs
print("Global margin is {0:.4f}, or {1:.2f}%".format(margin, margin*100))

Global margin is 0.2719, or 27.19%


#### 2) Can you think of other interesting metrics to optimize

Not in the csv: Click-through rate (CTR), Conversion rate.

I would also like to understand why sometimes it takes more seconds than others to get the money - some of them don't make us any money in 30 seconds, some of them do in 10 seconds. I guess it is very possible that we are losing money in some type of ads. Maybe we should only display them in certain platforms where people is willing to spend more time watching ads - and this is a metric we could get from our csv, the number of seconds per device.

Metrics to optimize, from the csv:
1) Profits - No explanation needed.
2) Profits per device - We want to make sure all devices are profitable.
3) Profits per OS - We want to make sure all OSs are profitable.


#### 3) What are the most profitable Operating Systems?

In [4]:
df_agg = df[['user_operating_system','cost', 'revenue']].groupby(
    ['user_operating_system']).sum()
series_profits = df_agg["revenue"]-df_agg["cost"]
print(series_profits[series_profits==max(series_profits)])

user_operating_system
iOS    115.521775
dtype: float64


## Machine learning questions

#### How would you use this historical data to predict the event 'we benefit from a revenue' (ie revenue > 0) in a dataset where the revenue is not known yet?

#### Compute the prediction accuracy of a well chosen algorithm and comment the results. Do not hesitate to describe your methodology.

The problem at hand is a problem of supervised learning, in particular, it is a regression problem. We need to build a regressor from our dataset, using:

- Four features: *'user_operating_system', 'user_device', 'average_seconds_played', 'cost'*. 
- A label: *revenues*.
- A regression algorithm.


##### Data cleaning:

We have a dataset with 100 000 examples - that is pretty good to use some of the more complex algorithms like SVMs with kernels, or ensemble learning (random forests, adaboost). Neural networks could also be considered but, considering the scope of this work, it would take too long to train and optimize.

The first problem we face is that the features user_device and average_seconds_played are not present in some examples of the data set - or, in pandas' jargon, they are NaNs.

In [5]:
feature_cols = ['user_operating_system', 'user_device', 
                'average_seconds_played', 'cost']
X = df.loc[:, feature_cols]

X.isnull().sum()

user_operating_system         0
user_device                   8
average_seconds_played    61828
cost                          0
dtype: int64

In the case of *user_device*, it is not a big problem (only 8 labels out of 100 000, we can discard 8 without any second thoughts). Also substituting the NaNs for the average or something similar is tricky since this is a categorical feature (i.e., a string) - we could use the most frequent value but, again, there is no need. 

In the case of *average_seconds_played*, the problem is different: more than half of the data does not contain a value for the feature. However in this case, this is a numerical feature which gives us two options:

1) Removal: We would end up with 39% of the dataset... do we really want to do this? Probably not, but hey, let's try and let the data talk.

2) Substituting the data with its average, mode or median. This looks more promising, and it will be our default approach.

##### Feature Selection and Scaling

Feature generation and feature selection is one of the most important time consumn
Feature scaling

In [6]:
print(len(df))

100000


In [7]:
 'user_operating_system', 'user_device',
       'average_seconds_played', 'cost', 'revenue'

IndentationError: unexpected indent (<ipython-input-7-05ae2b97a5af>, line 2)

In [ ]:
from sklearn import svm

X = [[0, 0], [2, 2]]
y = [0.5, 2.5]
clf = svm.SVR()
clf.fit(X, y) 


clf.predict([[1, 1]])



Since we don't have another dataset, let's use the given dataset as a proxy for the new dataset - although, of course, we will use the revenue not as a feature but as a label. 

Use an SVR and ensemble regressors 
Feature selection? Evidently, no creative id

Cross Validation